[<img src="https://colab.research.google.com/img/colab_favicon.ico" alt="Open this notebook in Google Colab" width="80">](https://colab.research.google.com/github/GVourvachakis/Stochastic_Gamma_distribution_aware_MDS/blob/main/distribution_aware_mds.ipynb)

In [ ]:
# Load domain libraries and dependencies
import numpy as np
#import matplotlib.pyplot as plt
from typing import List, Dict, Any
from time import time

# Load California Housing Dataset
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

X, y = fetch_california_housing(return_X_y=True)

# Split the California housing dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Output the shapes of the resulting datasets
print("California Housins Dataset:")
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)

# MDS and Linear Regression Analysis

$sampled \ events $ ~ $  uni\{1, dim(S)\}$, where $S = sample \ space$ (X.shape[0])

In [2]:
from sklearn.manifold import MDS
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler

In [3]:
# Initialize StandardScaler for feature scaling
scaler = StandardScaler()

# Scale the features
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [4]:
import warnings

def mds_regression_analysis(X_train, y_train, X_test, y_test, feature_dim: List[int]) -> Dict[int, Dict[str, float]] | None:
    """
    Perform MDS dimensionality reduction and linear regression analysis with memory optimization.

    Parameters:
    -----------
    X_train : array-like of shape (n_samples, n_features)
        Training data
    y_train : array-like of shape (n_samples,)
        Target values for training
    X_test : array-like of shape (n_samples, n_features)
        Test data
    y_test : array-like of shape (n_samples,)
        Target values for test
    feature_dim : List[int]
        List of dimensions to reduce to

    Returns:
    --------
    Dict : Dictionary containing results for each dimension
    """

    # Dictionary to store results
    results = {}

    # Suppress convergence warnings
    warnings.filterwarnings('ignore')

    try:
        for n_components in feature_dim:
            print(f"Processing {n_components}D reduction...")

            # Initialize MDS with memory-efficient parameters
            mds = MDS(
                n_components=n_components,
                random_state=42,
                n_init=1,              # Single initialization
                max_iter=300,          # Limited iterations
                n_jobs=1,             # Disable parallel processing
                dissimilarity='euclidean',  # Use euclidean distance (faster)
                eps=1e-3              # Slightly relaxed convergence criterion
            )

            # Process training data
            print("Transforming training data...")
            X_train_mds = mds.fit_transform(X_train)

            # Process test data
            print("Transforming test data...")
            X_test_mds = mds.fit_transform(X_test)

            # Train linear regression
            print("Training linear regression...")
            lr = LinearRegression()
            lr.fit(X_train_mds, y_train)

            # Make predictions
            print("Making predictions...")
            y_pred = lr.predict(X_test_mds)

            # Calculate metrics
            mse = mean_squared_error(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)

            # Store results
            results[n_components] = {
                'mse': mse,
                'mae': mae
            }

            # Clear some memory
            del X_train_mds, X_test_mds, y_pred

            print(f"Completed {n_components}D reduction")

        return results

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

In [5]:
# Run the analysis with a smaller subset if needed
def run_analysis(X_train, y_train, X_test, y_test, sample_size=None):
    """
    Run the analysis with optional data sampling for memory efficiency.

    Parameters:
    -----------
    sample_size : int, optional
        Number of samples to use. If None, uses full dataset.
    """
    feature_dims = [2, 3]

    start_time = time()

    if sample_size is not None and sample_size < len(X_train):
        # Random sampling for memory efficiency
        np.random.seed(42)
        train_idx = np.random.choice(len(X_train), sample_size, replace=False)
        X_train_subset = X_train[train_idx]
        y_train_subset = y_train[train_idx]

        test_idx = np.random.choice(len(X_test), sample_size//4, replace=False)
        X_test_subset = X_test[test_idx]
        y_test_subset = y_test[test_idx]

        print(f"Using subset of {sample_size} training samples and {sample_size//4} test samples")
        metrics = mds_regression_analysis(X_train_subset, y_train_subset,
                                       X_test_subset, y_test_subset, feature_dims)
    else:
        metrics = mds_regression_analysis(X_train, y_train, X_test, y_test, feature_dims)

    if metrics is not None:
        for dim in feature_dims:
            print(f"\nResults for {dim}D reduction:")
            print(f"MSE: {metrics[dim]['mse']:.4f}")
            print(f"MAE: {metrics[dim]['mae']:.4f}")

    print(f"Execution time: {time() - start_time:.2f} seconds")

    return metrics

In [6]:
# Try running with a smaller sample size first
# sample_size = 1000  # Adjust this value based on your available memory
# metrics = run_analysis(X_train_scaled, y_train, X_test_scaled, y_test, sample_size=sample_size)

## Sampling with distribution matching/reconstruction awareness

Data-specific observations from target's density histogram:

1. The distribution is right-skewed
2. There's a main peak around 1.5-2.0
3. There's a long tail extending to about 5.0
4. The distribution appears to be unimodal
5. The sampling needs to maintain proper representation across all price ranges

**Note**: *Kullback Leibler (KL) divergence* is generally more appropriate for comparing distributions that are similar in shape or location. In contrast, *Wasserstein distance* is robust to shifts in mass and can handle the shape and spread differences better, making it preferable for comparing distributions like the one described.

In [7]:
from utils import validate_sampling_distribution, perform_baseline_regression, uniform_sampling_mds_analysis

In [1]:
%pip install pyinstrument

Note: you may need to restart the kernel to use updated packages.


The cell below is expected to run around **~ 20 min**.
Employing 3 epochs/runs and obtaining profile statistics for the mds training routine.

In [ ]:
#Run the complete analysis
warnings.filterwarnings('ignore')
np.random.seed(42)

# Perform analysis
results = uniform_sampling_mds_analysis(X_train_scaled, y_train, X_test_scaled, y_test, sample_size=5_000, n_runs=1)

# Validate sampling distribution
validate_sampling_distribution(
                                  results['sampling_validation']['y_full'],
                                  results['sampling_validation']['y_sample'],
                                  title = "Target Variable Distribution: Full vs Sampled Dataset",
                                  dataset = "California Housing"
                              )

# Print comprehensive results
print("\nAnalysis Results:")
print("\nBaseline Metrics (No dimensionality reduction):")
print(f"MSE: {results['baseline_metrics']['mse']:.4f}")
print(f"MAE: {results['baseline_metrics']['mae']:.4f}")
print(f"R²: {results['baseline_metrics']['r2']:.4f}")

for dim in ['2D', '3D']:
    print(f"\n{dim} MDS Reduction Metrics:")
    metrics = results['performance_metrics'][dim]
    print(f"MSE: {metrics['mse_mean']:.4f} ± {metrics['mse_std']:.4f}")
    print(f"MAE: {metrics['mae_mean']:.4f} ± {metrics['mae_std']:.4f}")
    print(f"R²: {metrics['r2_mean']:.4f} ± {metrics['r2_std']:.4f}")

print("\nExecution Times:")
for key, value in results['timing_metrics'].items():
    print(f"{key}: {value:.2f} seconds")

# Single Run Uniform Sampling Benchmarks

### Baseline Metrics (No Dimensionality Reduction)
- **MSE**: 0.5559
- **MAE**: 0.5332
- **R²**: 0.5758

### 2D MDS Reduction Metrics
- **MSE**: 1.4021 ± 0.0527
- **MAE**: 0.9330 ± 0.0183
- **R²**: -0.0745 ± 0.0120

### 3D MDS Reduction Metrics
- **MSE**: 1.4745 ± 0.2832
- **MAE**: 0.9360 ± 0.1055
- **R²**: -0.1266 ± 0.1880

### Execution Times
- **2D**: 198.76 seconds
- **3D**: 184.78 seconds
- **Total**: 1124.21 seconds


# Preconditioned Mini-batch stratified MDS

In [8]:
from scipy import stats
from scipy.stats import ks_2samp, wasserstein_distance
from tqdm import tqdm
from sklearn.preprocessing import KBinsDiscretizer
from pyinstrument import Profiler

class DistributionAwareStratifiedSampler:
    def __init__(self, n_bins=5):
        self.n_bins = n_bins
        self.kbd = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='quantile')
        self.distribution_params = {}
        self.distributions_info = {}  # Store detailed distribution information

    def fit(self, y):
        """Fit the sampler to the target variable distribution."""
        self.y_binned = self.kbd.fit_transform(y.reshape(-1, 1)).flatten()
        self.bin_distributions = {}
        self.distributions_info = {}  # Reset distribution info

        # Get bin boundaries for reporting
        bin_edges = self.kbd.bin_edges_[0]

        # Fit distribution for each bin
        for bin_idx in range(self.n_bins):
            bin_data = y[self.y_binned == bin_idx]
            bin_range = f"[{bin_edges[bin_idx]:.2f}, {bin_edges[bin_idx+1]:.2f})"

            # Find best distribution for this bin
            best_dist, best_params, ks_stat = self._fit_distribution(bin_data)

            self.bin_distributions[bin_idx] = {
                'distribution': best_dist,
                'params': best_params
            }

            # Store detailed information about the fit
            if best_dist is not None:
                self.distributions_info[bin_idx] = {
                    'bin_range': bin_range,
                    'distribution_name': best_dist.name,
                    'parameters': best_params,
                    'ks_statistic': ks_stat,
                    'sample_size': len(bin_data)
                }
            else:
                self.distributions_info[bin_idx] = {
                    'bin_range': bin_range,
                    'distribution_name': 'fallback_uniform',
                    'parameters': None,
                    'ks_statistic': None,
                    'sample_size': len(bin_data)
                }

        return self

    def _fit_distribution(self, data):
        """Find the best-fitting distribution for the given data."""
        distributions = [
            (stats.gamma, ['a', 'loc', 'scale']),
            (stats.lognorm, ['s', 'loc', 'scale']),
            (stats.norm, ['loc', 'scale']),
            (stats.beta, ['a', 'b', 'loc', 'scale'])
        ]

        best_fit = None
        best_ks = float('inf')
        best_params = None

        for distribution, param_names in distributions:
            try:
                # Fit distribution
                params = distribution.fit(data)
                # Generate samples for KS test
                samples = distribution.rvs(*params, size=len(data))
                # Perform KS test
                ks_stat, _ = ks_2samp(data, samples)

                if ks_stat < best_ks:
                    best_ks = ks_stat
                    best_fit = distribution
                    best_params = dict(zip(param_names, params))
            except:
                continue

        return best_fit, best_params, best_ks if best_fit is not None else None

    def sample(self, y, sample_size):
        """Generate stratified samples using fitted distributions."""
        samples_per_bin = sample_size // self.n_bins
        sampled_indices = []

        for bin_idx in range(self.n_bins):
            bin_indices = np.where(self.y_binned == bin_idx)[0]
            bin_dist_info = self.bin_distributions[bin_idx]

            if bin_dist_info['distribution'] is None:
                # Fallback to random sampling if distribution fitting failed
                selected_indices = np.random.choice(
                    bin_indices,
                    size=min(samples_per_bin, len(bin_indices)),
                    replace=False
                )
            else:
                # Generate samples from fitted distribution
                dist = bin_dist_info['distribution']
                params = bin_dist_info['params']

                # Generate more samples than needed for better matching
                oversample_factor = 2
                generated_samples = dist.rvs(
                    **params,
                    size=samples_per_bin * oversample_factor
                )

                # Find closest matches in the original data
                bin_data = y[bin_indices]
                selected_indices = []

                for target_value in generated_samples[:samples_per_bin]:
                    closest_idx = bin_indices[
                        np.abs(bin_data - target_value).argmin()
                    ]
                    if closest_idx not in selected_indices:
                        selected_indices.append(closest_idx)

                # If we couldn't find enough unique matches, fall back to random sampling
                remaining = samples_per_bin - len(selected_indices)
                if remaining > 0:
                    available_indices = list(
                        set(bin_indices) - set(selected_indices)
                    )
                    if available_indices:
                        additional_indices = np.random.choice(
                            available_indices,
                            size=min(remaining, len(available_indices)),
                            replace=False
                        )
                        selected_indices.extend(additional_indices)

            sampled_indices.extend(selected_indices)

        return np.array(sampled_indices)

    def get_distribution_summary(self):
        """Return a formatted summary of the fitted distributions."""
        summary = "Distribution Fitting Summary:\n" + "="*50 + "\n"
        for bin_idx, info in self.distributions_info.items():
            summary += f"\nBin {bin_idx} ({info['bin_range']}):\n"
            summary += f"  Distribution: {info['distribution_name']}\n"
            summary += f"  Sample Size: {info['sample_size']}\n"
            if info['ks_statistic'] is not None:
                summary += f"  KS Statistic: {info['ks_statistic']:.4f}\n"
            if info['parameters'] is not None:
                summary += "  Parameters:\n"
                for param_name, param_value in info['parameters'].items():
                    summary += f"    {param_name}: {param_value:.4f}\n"
        return summary

In [9]:
def optimized_stratified_mds_analysis(X_train, y_train, X_test, y_test, \
                                      sample_size=5_000, n_runs = 3) -> Dict[str, Dict[str, Any]]:

    """
    Perform MDS analysis using distribution-aware stratified sampling with validation metrics.
    """

    results = {
        'sampling_validation': {},
        'performance_metrics': {'2D': {}, '3D': {}},
        'timing_metrics': {},
        'baseline_metrics': {},
        'distribution_info': {}  # New field for distribution information
    }

    start_time = time()

    # Initialize and fit the distribution-aware sampler
    sampler = DistributionAwareStratifiedSampler(n_bins=5)
    sampler.fit(y_train)

    # Store distribution information
    results['distribution_info']['summary'] = sampler.get_distribution_summary()
    results['distribution_info']['details'] = sampler.distributions_info

    results_2d = {'mse': [], 'mae': [], 'r2': []}
    results_3d = {'mse': [], 'mae': [], 'r2': []}

    # Initialize profiler
    profiler = Profiler()

    n_runs = n_runs
    for run in tqdm(range(n_runs), desc="Overall runs"):
        # Generate samples using the distribution-aware sampler
        train_indices = sampler.sample(y_train, sample_size)

        # Sample the data
        X_train_sample = X_train[train_indices]
        y_train_sample = y_train[train_indices]

        # Sample test set
        test_size = len(y_test) // 4
        test_indices = np.random.choice(len(y_test), size=test_size, replace=False)
        X_test_sample = X_test[test_indices]
        y_test_sample = y_test[test_indices]

        # Store sampling validation data for first run
        if run == 0:
            results['sampling_validation'] = {
                'y_full': y_train,
                'y_sample': y_train_sample,
                'sample_size': len(y_train_sample)
            }

        # Profile MDS transformation for both 2D and 3D
        if profiler.is_running:
            profiler.stop() # maintaining asynchronous mode
        profiler.start()  # Start profiling

        # Process both 2D and 3D reductions
        for n_components in [2, 3]:
            dim_start_time = time()
            tqdm.write(f"Run {run + 1}/{n_runs}: Processing {n_components}D reduction...")

            # MDS transformation
            mds = MDS(
                n_components=n_components,
                random_state= 42 + run, # switch between pseudo-random microstates
                n_init=1,
                max_iter=300,
                n_jobs=-1, # multi-core execution [use all available CPU cores]
                dissimilarity='euclidean',
                eps=1e-3
            )

            # Transform data
            X_train_mds = mds.fit_transform(X_train_sample)
            X_test_mds = mds.fit_transform(X_test_sample)

            # Fit regression and predict
            lr = LinearRegression()
            lr.fit(X_train_mds, y_train_sample)
            y_pred = lr.predict(X_test_mds)

            # Calculate metrics
            mse = mean_squared_error(y_test_sample, y_pred)
            mae = mean_absolute_error(y_test_sample, y_pred)
            r2 = r2_score(y_test_sample, y_pred)

            # Store results
            metrics_dict = results_2d if n_components == 2 else results_3d
            metrics_dict['mse'].append(mse)
            metrics_dict['mae'].append(mae)
            metrics_dict['r2'].append(r2)

            # Store timing for first run
            if run == 0:
                results['timing_metrics'][f'{n_components}D'] = time() - dim_start_time

        profiler.stop()  # Stop profiling

    # Print profiling results
    profiler.print()

    # Calculate baseline metrics
    print("\nCalculating baseline metrics...")
    results['baseline_metrics'] = perform_baseline_regression(X_train, y_train, X_test, y_test) # by "utils" module


    # Calculate and store final performance metrics
    for dim, res in [('2D', results_2d), ('3D', results_3d)]:
        results['performance_metrics'][dim] = {
            'mse_mean': np.mean(res['mse']),
            'mse_std': np.std(res['mse']),
            'mae_mean': np.mean(res['mae']),
            'mae_std': np.std(res['mae']),
            'r2_mean': np.mean(res['r2']),
            'r2_std': np.std(res['r2'])
        }

    # Store total execution time
    results['timing_metrics']['total'] = time() - start_time

    # Print distribution summary
    #print("\nDistribution Fitting Results:")
    print(results['distribution_info']['summary'])

    # Access detailed distribution parameters for specific bins
    for bin_idx, info in results['distribution_info']['details'].items():
        print(f"\nBin {bin_idx} distribution: {info['distribution_name']}")

    return results

The cell below is expected to run around **~ 20 min**.
Employing 3 epochs/runs, obtaining profile statistics for the mds training routine, running for the optimal sampling distribution of each bin.

In [ ]:
# Perform analysis
results = optimized_stratified_mds_analysis(X_train, y_train, X_test, y_test, n_runs=1)

## **Characterization of Gamma-Beta distribution and KS score**

**Distribution: Beta**: Indicates that the beta distribution is the underlying statistical model used to fit the data.

**Sample Size**: Specifies the number of observations or data points used to fit the beta distribution.

**KS Statistic**: The Kolmogorov-Smirnov (KS) statistic measures the maximum distance between the empirical cumulative distribution function (CDF) of the sample data and the CDF of the fitted beta distribution. A smaller KS statistic suggests that the beta distribution is a good fit for the data.

Parameters:
* **a**: The first shape parameter of the beta distribution, which influences the distribution's skewness and kurtosis.

* **b**: The second shape parameter of the beta distribution, similarly affecting its shape.

* **loc**: The location parameter, which shifts the distribution along the x-axis. In this case, it adjusts the lower bound of the beta distribution.

* **scale**: The scale parameter, which stretches or compresses the distribution, determining the width of the range over which the beta distribution is defined.

In [ ]:
# Suppress warnings
#warnings.filterwarnings('ignore')  # Ignore warnings for a cleaner output
np.random.seed(42)  # Ensure reproducibility

# Validate the sampling distribution (by "utils" module)
validate_sampling_distribution(
    y_full=results['sampling_validation'].get('y_full', []),
    y_sample=results['sampling_validation'].get('y_sample', []),
    title="Target Variable Distribution: Full vs Sampled Dataset",
    dataset = "California Housing"
)

# Display comprehensive analysis results
print("\nAnalysis Results:")

# Print baseline metrics for comparison (without dimensionality reduction)
baseline_metrics = results.get('baseline_metrics', {})

print("\nBaseline Metrics (No dimensionality reduction):")
print(f"MSE: {baseline_metrics.get('mse', float('nan')):.4f}")
print(f"MAE: {baseline_metrics.get('mae', float('nan')):.4f}")
print(f"R²: {baseline_metrics.get('r2', float('nan')):.4f}")

# Print performance metrics for each MDS reduction (2D and 3D)
for dim in ['2D', '3D']:
    print(f"\n{dim} MDS Reduction Metrics:")
    performance_metrics = results['performance_metrics'].get(dim, {})
    print(f"MSE: {performance_metrics.get('mse_mean', float('nan')):.4f} ± {performance_metrics.get('mse_std', float('nan')):.4f}")
    print(f"MAE: {performance_metrics.get('mae_mean', float('nan')):.4f} ± {performance_metrics.get('mae_std', float('nan')):.4f}")
    print(f"R²: {performance_metrics.get('r2_mean', float('nan')):.4f} ± {performance_metrics.get('r2_std', float('nan')):.4f}")

# Print execution times for dimensionality reductions and the total analysis
print("\nExecution Times:")
timing_metrics = results.get('timing_metrics', {})
for key, value in timing_metrics.items():
    print(f"{key}: {value:.2f} seconds")